# Preprocessing Data

In this notebook we will produce the  model input data.  
For that, we will  **clean** the data and **enrich**  it, and then **extract** features from raw data.  
This includes:  
1. Cleaning the questions / answers (removing stop words, tokenizing)
2. Enrichment: Marking diagnosis using thumb rules (Note: Eventually, we did not use this data)  
3. Enrichment: Adding a question category to data (given in train / validation sets, thumb rules + prediction to test set)
4. Pre processing: Getting Embedding for questions (get_text_features)  
    For this, we used Spacy's NLP package

### Some main functions we used:

In [ ]:
import IPython
from common.functions import get_highlighted_function_code

#### get_text_features for getting embedding of text

In [2]:
from pre_processing.prepare_data import get_text_features
code = get_highlighted_function_code(get_text_features,remove_comments=True)
IPython.display.display(code)

#### pre_process_raw_data for the data pre processing:

In [3]:
from pre_processing.prepare_data import  pre_process_raw_data
code = get_highlighted_function_code(pre_process_raw_data,remove_comments=True)
IPython.display.display(code)

#### Cleaning the data:

In [7]:
# from pre_processing.data_cleaning import clean_data
# code = get_highlighted_function_code(clean_data,remove_comments=True)
# IPython.display.display(code)

#### Enriching the data

In [6]:
# from pre_processing.data_enrichment import enrich_data
# code = get_highlighted_function_code(enrich_data,remove_comments=True)
# IPython.display.display(code)

---
## The code:

In [8]:
# %%capture
from common.settings import get_nlp, data_access
from common.functions import get_image,  get_size
from pre_processing.prepare_data import get_text_features, pre_process_raw_data
from pre_processing.data_enrichment import enrich_data
from pre_processing.data_cleaning import clean_data
from common.utils import VerboseTimer
from collections import Counter
import os
from pandas import HDFStore
import pyarrow as pa
import pyarrow.parquet as pq
import logging
from pathlib import Path

In [9]:
logger = logging.getLogger(__name__)

##### Getting the nlp engine
(doing it once - it is a singleton)

In [10]:
nlp = get_nlp()

[2019-05-03 11:27:55][common.settings][DEBUG] using embedding vector: en_core_web_lg
[2019-05-03 11:27:56][common.settings][DEBUG] Got NLP engine (en_core_web_lg)


Getting the raw input

In [11]:
image_name_question = data_access.load_raw_input()

[2019-05-03 11:28:00][data_access.api][DEBUG] Loading data from: C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\raw_data.h5
[2019-05-03 11:28:00][common.utils][DEBUG] Starting 'Loading raw data'
[2019-05-03 11:28:00][common.utils][DEBUG] Loading raw data: 0:00:00.093055


In [12]:
image_name_question.head()

,image_name,question,answer,group,path
0,synpic41148,what kind of image is this?,cta - ct angiography,train,C:\Users\Public\Documents\Data\2019\train\Trai...
1,synpic43984,is this a t1 weighted image?,no,train,C:\Users\Public\Documents\Data\2019\train\Trai...
2,synpic38930,what type of imaging modality is used to acqui...,us - ultrasound,train,C:\Users\Public\Documents\Data\2019\train\Trai...
3,synpic52143,is this a noncontrast mri?,no,train,C:\Users\Public\Documents\Data\2019\train\Trai...
4,synpic20934,what type of image modality is this?,xr - plain film,train,C:\Users\Public\Documents\Data\2019\train\Trai...


## Clean and enrich the data

In [13]:
orig_image_name_question = image_name_question.copy()
image_name_question = clean_data(image_name_question)
image_name_question = enrich_data(image_name_question)

Looking for word: arch: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:03<00:00, 26.74it/s]


In [14]:
groups = image_name_question.groupby('group')
groups.describe()

answer                     diagnosis                   image_name  \
            count unique    top  freq     count unique top   freq      count   
group                                                                          
test            0      0    NaN   NaN       500      1        500        500   
train       12792   1552  axial  1558     12792    274      10569      12792   
validation   2000    470  axial   213      2000    133       1669       2000   

                   ...                               processed_question       \
           unique  ...                                              top freq   
group              ...                                                         
test          500  ...   what is the primary abnormality in this image?   24   
train        3200  ...           what abnormality is seen in the image?  776   
validation    500  ...           what abnormality is seen in the image?  133   

           question                                                         \
              count unique                                             top   
group                                                                        
test            500    138  what is the primary abnormality in this image?   
train         12792    247          what abnormality is seen in the image?   
validation     2000    186          what abnormality is seen in the image?   

                question_category                      
           freq             count unique    top  freq  
group                                                  
test         24                 0      0    NaN   NaN  
train       776             12792      4  Organ  3200  
validation  133              2000      4  Plane   500  

[3 rows x 32 columns]

In [15]:
image_name_question.head()
image_name_question.sample(n=4)

,image_name,question,answer,group,path,processed_question,processed_answer,diagnosis,question_category
11403,synpic28559,what is abnormal in the ct scan?,nonspecific interstitial pneumonitis,train,C:\Users\Public\Documents\Data\2019\train\Trai...,what is abnormal ct?,nonspecific interstitial pneumonitis,,Abnormality
7997,synpic27235,what is one organ system seen in this image?,genitourinary,train,C:\Users\Public\Documents\Data\2019\train\Trai...,what is one organ system seen in this image?,genitourinary,,Organ
12282,synpic40485,what is the primary abnormality in this image?,multiple sclerosis,train,C:\Users\Public\Documents\Data\2019\train\Trai...,what is the primary abnormality in this image?,multiple sclerosis,multiple sclerosis,Abnormality
5088,synpic17010,what plane is this?,axial,train,C:\Users\Public\Documents\Data\2019\train\Trai...,what plane is this?,axial,,Plane


## Do the actual pre processing

#### If running in an exported notebook, use the following:
(indent everything to be under the main guard) - for avoiding recursive spawning of processes

In [14]:
from multiprocessing import freeze_support
if __name__ == '__main__':
    print('in main')
    freeze_support()

in main


Note:  
This might take a while...

In [17]:
logger.debug('----===== Preproceccing train data =====----')
image_name_question_processed = pre_process_raw_data(image_name_question)

[2019-05-03 11:31:18][__main__][DEBUG] ----===== Preproceccing train data =====----
[2019-05-03 11:31:18][common.utils][DEBUG] Starting 'Pre processing'
[2019-05-03 11:31:19][pre_processing.prepare_data][INFO] Answer: removing stop words and tokenizing
[2019-05-03 11:31:19][common.utils][DEBUG] Starting 'Answer Tokenizing'
[2019-05-03 11:31:19][common.utils][DEBUG] Answer Tokenizing: 0:00:00.085901
[2019-05-03 11:31:19][pre_processing.prepare_data][INFO] Question: removing stop words and tokenizing
[2019-05-03 11:31:19][common.utils][DEBUG] Starting 'Question Tokenizing'
[2019-05-03 11:31:19][common.utils][DEBUG] Question Tokenizing: 0:00:00.151748
[2019-05-03 11:31:19][pre_processing.prepare_data][INFO] Getting answers embedding
[2019-05-03 11:31:19][common.utils][DEBUG] Starting 'Answer Embedding'
[2019-05-03 11:32:17][common.utils][DEBUG] Answer Embedding: 0:00:58.039930
[2019-05-03 11:32:17][pre_processing.prepare_data][INFO] Getting questions embedding
[2019-05-03 11:32:17][common

In [18]:
image_name_question_processed.sample(2)

,image_name,question,answer,group,path,processed_question,processed_answer,diagnosis,question_category,answer_embedding,question_embedding
277,synpic57196.jpg,how was the image taken?,mr - flair,train,C:\Users\Public\Documents\Data\2019\train\Trai...,how was the image taken,mr flair,,Modality,"[1.4048264026641846, -1.276012659072876, 0.128...","[1.392259955406189, 1.1085478067398071, 0.2110..."
13259,synpic44506.jpg,is this a t2 weighted image?,no,validation,C:\Users\Public\Documents\Data\2019\validation...,is this t2 weighted image,no,,Modality,"[0.029011979699134827, 1.9719411134719849, 1.5...","[1.4450395107269287, 0.4569704532623291, -3.04..."


Take a look at data of a single image:

In [17]:
image_name_question_processed[image_name_question_processed.image_name == 'synpic52143.jpg'].head()

,image_name,question,answer,group,path,processed_question,processed_answer,diagnosis,question_category,answer_embedding,question_embedding
3,synpic52143.jpg,is this a noncontrast mri?,no,train,C:\Users\Public\Documents\Data\2019\train\Trai...,is this noncontrast mri,no,,Modality,"[0.029011979699134827, 1.9719411134719849, 1.5...","[1.2045111656188965, 0.6815400123596191, -3.26..."
3203,synpic52143.jpg,which plane is the image shown in?,coronal,train,C:\Users\Public\Documents\Data\2019\train\Trai...,which plane is the image shown in,coronal,,Plane,"[-2.5162551403045654, -0.6533107757568359, 0.8...","[-2.4232277870178223, 4.579081058502197, 0.132..."
6403,synpic52143.jpg,the mri shows what organ system?,spine and contents,train,C:\Users\Public\Documents\Data\2019\train\Trai...,the mri shows what organ system,spine and contents,,Organ,"[3.57601261138916, 2.5560226440429688, 1.97663...","[2.0428223609924316, 0.2528434097766876, -1.45..."
9603,synpic52143.jpg,what is the primary abnormality in this image?,bone tumor/ chordoma,train,C:\Users\Public\Documents\Data\2019\train\Trai...,what is the primary abnormality in this image,bone tumor chordoma,bone tumor,Abnormality,"[1.3663643598556519, 0.21053718030452728, -2.3...","[-2.6657872200012207, 1.1844078302383423, 0.02..."


In [19]:
from collections import Counter

How many categories did we get for questions?

In [21]:
print('--Test--')
print(Counter(image_name_question_processed[image_name_question_processed.group=='test'].question_category.values))
print('--All--')
print(Counter(image_name_question_processed.question_category.values))

--Test--
Counter({'Organ': 126, 'Modality': 125, 'Plane': 125, 'Abnormality': 114, 'Abnormality_yes_no': 10})
--All--
Counter({'Organ': 3826, 'Modality': 3825, 'Plane': 3825, 'Abnormality': 3673, 'Abnormality_yes_no': 143})


#### Saving the data, so later on we don't need to compute it again

In [18]:
saved_path = data_access.save_processed_data(image_name_question_processed)

[2019-04-22 14:06:31][data_access.api][DEBUG] Saving the processed data to:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-04-22 14:06:31][common.utils][DEBUG] Starting 'Saving processed data'
[2019-04-22 14:06:45][common.utils][DEBUG] Saving processed data: 0:00:14.165711


In [19]:
print(f'Data saved at:\n{saved_path}')

Data saved at:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
